In [1]:
import os
import torch
import scipy.io
import matplotlib.pyplot as plt
import cvxpy as cp
import seaborn as sns
import numpy as np

os.chdir('..')
import data_processing.preprocessing as preprocessing
from config import left_cut, right_cut
from utils import beerlamb_multi

sns.set()

In [ ]:
img = scipy.io.loadmat('dataset/LWP483_10Jan2017_SharedHyperProbe.mat')

### Visualising spectrograms with calibration
intensity1_c = []
intensity2_c = []

wavelengths = img['wavelengths'].astype(float)
white_full = img['refSpectrum'].astype(float)
dark_full = img['DarkCount'].astype(float)
spectr = img['spectralDatameasured'].astype(float)

idx = (wavelengths >= left_cut) & (wavelengths <= right_cut)
wavelengths = wavelengths[idx]
spectr = spectr[idx.squeeze()]
dark_full = dark_full[idx.squeeze()]
white_full = white_full[idx.squeeze()]

# We wanna calibrate our HSI images w.r.t. white reference. Why? Because we saw by looking at the raw data that different
# wavelengths have different reflection from the white materila (i.e. reflecting 99% of light). So
# we calibrate our HSI images
print(white_full.shape, dark_full.shape, wavelengths.shape, spectr.shape)

fig, (ax, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))

spectr_1 = (spectr[:, 0] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0])
ax.plot(wavelengths, spectr_1)
#ax.plot(wavelengths, spectr)
ax.set_xlabel("Wavelength", fontsize=15)
ax.set_title("Base Spectrogram", fontsize=15)
ax2.set_xlim(left_cut, right_cut)
ax.set_ylim(-0.01, 0.3)

i = 100  #7830
spectr_2 = (spectr[:, i] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0])
ax1.plot(wavelengths, spectr_2)
ax1.set_xlabel("Wavelength", fontsize=15)
#ax1.plot(wavelengths, spectr[:,100])
#ax.set_xlabel("Wavelength", fontsize=20)
ax1.set_title("Hypoxia Spectrogram", fontsize=15)
ax2.set_xlim(left_cut, right_cut)
ax1.set_ylim(-0.01, 0.3)

spectr_1[spectr_1 <= 0] = 0.0001
spectr_2[spectr_2 <= 0] = 0.0001
spectr_3 = spectr_2 / spectr_1
ax2.plot(wavelengths, spectr_3)
ax2.axhline(y=0.0, color='r', linestyle='-')
ax2.set_xlabel("Wavelength", fontsize=15)
ax2.set_title("Diff Spectrogram", fontsize=15)
ax2.set_xlim(left_cut, right_cut)
#ax2.set_ylim(-0.3, 0.3)

In [ ]:
def optimisation(spectr1, spectr2):
    m = 4  # number of parameters (from 2 to 6)
    #np.random.seed(1)
    molecules, x = preprocessing.read_molecules(left_cut, right_cut, wavelengths)
    y_hbo2_f, y_hb_f, y_coxa, y_creda, _, _ = molecules
    M = np.transpose(np.vstack((np.asarray(y_hbo2_f),
                                np.asarray(y_hb_f),
                                np.asarray(y_coxa),
                                np.asarray(y_creda))))
    
    b = spectr2 / spectr1
    b = np.log(1 / np.asarray(b))  # see the writting above (we took log of I_R and there was also minus that went to the degree of the logarithm)
    X = cp.Variable((m, len(b)))
    b = np.swapaxes(b, 0, 1)
    
    print(M.shape, X.shape, b.shape)
     
    objective = cp.Minimize(cp.sum_squares(M @ X - b))
    constraints = [cp.abs(X[2]+X[3])<=0.01]
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    for i in range(len(b[0])):        
        if i % 1000 == 0:
            fig, (ax) = plt.subplots(ncols=1, figsize=(12, 4))
            fig.suptitle("Diff to timestep " + str(i), fontsize=16, y=1.05)
            Xi = -X[:,i]
            bi = b[:,i]

            err = np.asarray(np.log(beerlamb_multi(molecules, x, Xi.value, left_cut))) - bi
            err = np.mean(np.abs(err))
            ax.plot(x, np.log(beerlamb_multi(molecules, x, Xi.value, left_cut)), color='b', label='Predicted')
            ax.plot(x, bi, color='r', label='Real')
            ax.set_xlabel(str(np.around(Xi.value,decimals=4)), fontsize=12)
            ax.set_title("base", fontsize=12)
            ax.set_xlim(left_cut, right_cut)
            ax.annotate("Error: " + str(np.round(err, 4)),
                        xy=(0.6, 0.7),  # Coordinates for the annotation arrow point
                        xycoords='axes fraction')
            ax.legend()            
            fig.show()

    return -X.value, b

In [ ]:
cut = 7830

ref_spectr = (spectr[:, 0] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0])
ref_spectr[ref_spectr <= 0] = 0.0001

spectra_list = []
coef_list = []

#if i not in [100,200,400,2000]: continue
comp_spectr = np.array([(spectr[:, i] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0]) for i in range(1,cut+1)])
comp_spectr[comp_spectr <= 0] = 0.0001

coef_diff, spect_diff = optimisation(ref_spectr, comp_spectr)

spectra_list.append(spect_diff)
coef_list.append(coef_diff)

In [ ]:
utils.save_data(ref_spectr, spectra_list, coef_list)